In [1]:
import lasagne
import theano
import theano.tensor as T
import agentnet

WARNING (theano.configdefaults): Only clang++ is supported. With g++, we end up with strange g++/OSX bugs.


In [2]:
import numpy as np
class ModRelu(lasagne.layers.Layer):
    def __init__(self, incoming, b=lasagne.init.Uniform(range=0.01), **kwargs):
        super(ModRelu, self).__init__(incoming, **kwargs)
        print(self.input_shape)
        self.n_hidden = self.input_shape[-1] // 2
        self.hb = self.add_param(b, (self.n_hidden,), name='hb', regularizable=False, trainable=True)

    def get_output_for(self, input, **kwargs):
        print("Inside a ModReLU")
        input_flattened = input.reshape((-1, input.shape[-1]))

        swap_re_im = np.concatenate((np.arange(self.n_hidden, 2*self.n_hidden), np.arange(self.n_hidden)))
        modulus = T.sqrt(input_flattened**2 + input_flattened[:, swap_re_im]**2+1e-5)
        rescale = T.maximum(modulus + T.tile(self.hb, [2]).dimshuffle('x', 0), 0.) / (modulus + 1e-5)
        out = (input_flattened * rescale).reshape(input.shape)
        return out

In [7]:
from lasagne.layers import *
inp =T.matrix("inp")
l_in = InputLayer([None,2],input_var=inp)
l_mrelu = ModRelu(l_in)

out = get_output(l_mrelu)
f = theano.function([inp], T.grad(out.mean(),inp))

[None, 2]
Inside a ModReLU


'float64'

In [50]:
inp = input = T.matrix("inp")
n_hidden=1
epsilon=1e-5
hb = l_mrelu.hb #theano.shared(np.random.uniform(-0.01,0.01,size=(n_hidden,)).astype(theano.config.floatX))
#layer
input_flat = input
input = input.reshape((input.shape[0],2,n_hidden))

modulus_sqr= T.sum(input**2,1,keepdims=True) + epsilon
modulus = T.sqrt(modulus_sqr)
#modulus = T.switch(T.lt(modulus_sqr,epsilon),0,modulus)

rescale = T.maximum((modulus+hb[None,None,:])/(modulus+epsilon), 0.)

out = input *rescale
out = out.reshape(input_flat.shape)
#/layer

In [51]:
f = theano.function([inp], [T.grad(modulus.mean(),inp),modulus])

In [52]:
f(np.zeros([5,2]))

[array([[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]]), array([[[ 0.00316228]],
 
        [[ 0.00316228]],
 
        [[ 0.00316228]],
 
        [[ 0.00316228]],
 
        [[ 0.00316228]]])]

In [27]:
from layers import UnitaryLayer
from lasagne.layers import *
class step:
    n_inp = 2
    n_hid = 100

    l_in = InputLayer((None,n_inp))
    dense_input_layer = DenseLayer(l_in,n_hid*2,nonlinearity=None)

    l_prev_state = InputLayer((None,n_hid*2))
    unitary_step = UnitaryLayer(l_prev_state)

    added = ElemwiseSumLayer([dense_input_layer,unitary_step])

    new_state = ModRelu(added)
    
    out_layer = DenseLayer(new_state,1,nonlinearity=None)

(None, 200)


In [28]:
l_in = InputLayer([None,None,step.n_inp])

In [32]:
from agentnet.agent import Recurrence

rec = Recurrence(state_variables={step.new_state: step.l_prev_state},
                 input_sequences={step.l_in:l_in},
                 tracked_outputs=[step.out_layer],
                 unroll_scan=False)

/Users/cnst/anaconda/lib/python3.5/site-packages/agentnet/agent/recurrence.py:228: UserWarning: You are giving Recurrence an input sequence of undefined length (None).
Make sure it is always above <unspecified>(n_steps) you specified for recurrence
  "Make sure it is always above {}(n_steps) you specified for recurrence".format(n_steps or "<unspecified>"))


In [47]:
predictions_seq_layer = rec.get_sequence_layers()[1][0]
predicted_values = get_output(predictions_seq_layer)

Inside a ModReLU


In [40]:
predict = theano.function([l_in.input_var],predicted_values)

Inside a ModReLU


In [53]:
target_values = T.vector('target_output')
cost = T.mean((predicted_values - target_values)**2)
from utils.optimizations import custom_sgd

man = step.unitary_step.manifold
manifolds = {man.str_id : man}

params =get_all_params(predictions_seq_layer)

train = theano.function([l_in.input_var,target_values],cost,updates = custom_sgd(cost,params, 1e-4, manifolds))

TypeError: ('The following error happened while compiling the node', forall_inplace,cpu,grad_of_scan_fn}(Shape_i{1}.0, Alloc.0, Subtensor{int64:int64:int64}.0, Subtensor{int64:int64:int64}.0, Alloc.0, Subtensor{::int64}.0, Alloc.0, Alloc.0, Alloc.0, Alloc.0, Shape_i{1}.0, Shape_i{1}.0, W, Subtensor{int64}.0, Subtensor{int64}.0, InplaceDimShuffle{x,0}.0, InplaceDimShuffle{1,0}.0, InplaceDimShuffle{1,0}.0, InplaceDimShuffle{1,0}.0, Alloc.0, MakeVector{dtype='int64'}.0, InplaceDimShuffle{x,0}.0), '\n', 'Non-unit value on shape on a broadcastable dimension.', (0, 0), (False, True), 'Container name "None"')

In [43]:
def gen_data(min_length=50, max_length=51, n_batch=10):
    '''
    Generate a batch of sequences for the "add" task, e.g. the target for the
    following

    ``| 0.5 | 0.7 | 0.3 | 0.1 | 0.2 | ... | 0.5 | 0.9 | ... | 0.8 | 0.2 |
      |  0  |  0  |  1  |  0  |  0  |     |  0  |  1  |     |  0  |  0  |``

    would be 0.3 + .9 = 1.2.  This task was proposed in [1]_ and explored in
    e.g. [2]_.

    Parameters
    ----------
    min_length : int
        Minimum sequence length.
    max_length : int
        Maximum sequence length.
    n_batch : int
        Number of samples in the batch.

    Returns
    -------
    X : np.ndarray
        Input to the network, of shape (n_batch, max_length, 2), where the last
        dimension corresponds to the two sequences shown above.
    y : np.ndarray
        Correct output for each sample, shape (n_batch,).
    mask : np.ndarray
        A binary matrix of shape (n_batch, max_length) where ``mask[i, j] = 1``
        when ``j <= (length of sequence i)`` and ``mask[i, j] = 0`` when ``j >
        (length of sequence i)``.

    References
    ----------
    .. [1] Hochreiter, Sepp, and Jürgen Schmidhuber. "Long short-term memory."
    Neural computation 9.8 (1997): 1735-1780.

    .. [2] Sutskever, Ilya, et al. "On the importance of initialization and
    momentum in deep learning." Proceedings of the 30th international
    conference on machine learning (ICML-13). 2013.
    '''
    # Generate X - we'll fill the last dimension later
    X = np.concatenate([np.random.uniform(size=(n_batch, max_length, 1)),
                        np.zeros((n_batch, max_length, 1))],
                       axis=-1)
    mask = np.zeros((n_batch, max_length))
    y = np.zeros((n_batch,))
    # Compute masks and correct values
    for n in range(n_batch):
        # Randomly choose the sequence length
        length = np.random.randint(min_length, max_length)
        # Make the mask for this sample 1 within the range of length
        mask[n, :length] = 1
        # Zero out X after the end of the sequence
        X[n, length:, 0] = 0
        # Set the second dimension to 1 at the indices to add
        X[n, np.random.randint(length/10), 1] = 1
        X[n, np.random.randint(length/2, length), 1] = 1
        # Multiply and sum the dimensions of X to get the target value
        y[n] = np.sum(X[n, :, 0]*X[n, :, 1])
    # Center the inputs and outputs
    X -= X.reshape(-1, 2).mean(axis=0)
    y -= y.mean()
    return (X.astype(theano.config.floatX), y.astype(theano.config.floatX),
            mask.astype('int8'))

In [44]:
predict(gen_data()[0])

array([[[ -6.92199105e-03],
        [ -1.97507428e-02],
        [  1.74082346e-02],
        [  7.42347221e-03],
        [ -2.19017882e-01],
        [ -8.67836482e-04],
        [  8.81435102e-02],
        [  4.23115938e-01],
        [ -1.13449303e-01],
        [ -4.42030841e-01],
        [ -5.60414894e-01],
        [  1.62660729e-01],
        [ -1.20688536e-01],
        [  7.44984617e-01],
        [ -2.70578416e-01],
        [ -1.17051409e+00],
        [  1.11047347e+00],
        [  3.60285927e-01],
        [  7.19354561e-01],
        [ -6.32025527e-01],
        [ -1.54389300e+00],
        [  1.84505313e-01],
        [  2.16735671e+00],
        [  1.39620174e-01],
        [  3.34565856e-01],
        [ -1.33875794e+00],
        [ -1.72908872e+00],
        [  2.82335794e+00],
        [ -5.68532052e-01],
        [ -7.71857395e-01],
        [  2.69739180e+00],
        [ -4.46903684e+00],
        [  4.17401297e+00],
        [  1.09102467e+00],
        [ -6.83753321e+00],
        [  9.8874694